# CT-CBM 

---
Goal of the notebook: .

Inputs of the notebook:
- 

Output of the notebook:
- prediction + interpretation layer for text classification.


Takeaways: 
- .
- .

In [1]:
# !pip install -r requirements.txt

In [2]:
import sys
sys.path.append('../../run_experiments/')
sys.path.append('../../run_experiments/scripts')
sys.path.append('../../run_experiments/models')
sys.path.append('../../run_experiments/data')

import time
from tqdm import tqdm
import numpy as np
import pandas as pd

import torch
import torch.nn as nn
from datasets import load_dataset
from sklearn.metrics import f1_score
from torch.optim.lr_scheduler import StepLR
from torch.utils.data import DataLoader, Dataset, Subset


# model for CBM
# import fonction for getting PLM and tokenizer
from models.utils import load_model_and_tokenizer

from concepts_discovery_utils import extract_target_words, create_context_window, load_model, run_concepts_discovery


**Delete awkward warning**

In [3]:
import logging
loggers = [logging.getLogger(name) for name in logging.root.manager.loggerDict]
for logger in loggers:
    if "transformers" in logger.name.lower():
        logger.setLevel(logging.ERROR)

# 0. autoreload

In [4]:
#code for autoreload script associated with jupyter notebook
%load_ext autoreload
%autoreload 2

# 1.SETUP ENVIRONMENT VARIABLES

In [5]:
# import config
from load_config import load_config

model_name = 'bert-base-uncased'    # 'bert-base-uncased' ou 'deberta-large' or 'gemma'
dataset    = 'agnews'               # 'movies' / 'agnews' / 'dbpedia' / 'medical'/ 'ledgar'/ n24news
annotation = 'C3M'       # 'C3M' ou 'our_annotation' ou 'combined_annotation'
config = load_config(model_name, dataset)
config.annotation = annotation

# 2. Data Loading

In [7]:
# # import the data
from prepare_data import load_fc_prepare_data
prepare_data = load_fc_prepare_data(config.DATASET)
train_loader, test_loader, val_loader, train_df, val_df, test_df = prepare_data(config)

Chargement des fichiers sauvegardés...


# 3. Launching experimentation

In [8]:
import os
import torch
import torch.nn as nn
import numpy as np
from tqdm import tqdm
from sklearn.metrics import f1_score
from transformers import pipeline
from concepts_discovery_utils import (
    extract_target_words, create_context_window,
    load_model, run_concepts_discovery, calculate_macro_concept_frequencies,
    update_concept_frequencies, find_most_frequent_macro_concept,
    
 ) #summarize_concepts, 
from attribution_utils import (
    process_data_in_batches, get_example, example_attribution,
    split_dataloader
)
import json

# Ranking concepts 
from ranking_utils import rank_macro_concepts, most_k_important_macro_concepts, get_concept_at_rank, randomize_scores


# Launch experimentation for OUR pipeline

In [9]:
if config.model_name == 'gemma':
    from full_pipeline_gemma import JointResidualFittingModel
    from models.jointCBMv2_gemma import JointModel
else:
    from full_pipeline import JointResidualFittingModel
    from models.jointCBMv2 import JointModel

### 1. Load the joint model

In [10]:
embedder_model, embedder_tokenizer, ModelXtoCtoY_layer, _ = load_model_and_tokenizer(config, n_concepts = 1)
CBM_joint = JointModel(embedder_model, embedder_tokenizer, ModelXtoCtoY_layer, config, train_loader, val_loader)


### 2.call of the residual fitting part for pipeline below


In [11]:
from models.utils import RidgeLinearLayer

# residual fitting part for pipeline below
linear_layer = RidgeLinearLayer(config.dim, config.num_labels, l2_lambda=0.01)
linear_layer.to(config.device)

CBM_joint.linear_layer = linear_layer

### 3. Instancie the pipeline class

In [12]:
joint_residual_model = JointResidualFittingModel(
    joint_model = CBM_joint,
    linear_layer = linear_layer, 
    discovery_model = None, 
    discovery_tokenizer = None,
    config = config
    )

### 4. launch the pipeline

##  new dataset C3M

In [13]:
# # import the dataframe
from prepare_data import prepare_data_from_csv
df_aug_train, df_aug_test = prepare_data_from_csv(annotation  = config.annotation, config = config, return_test = True)

# # dataloaders
from concepts_bank_utils import create_dataloader
notre_loader_train = create_dataloader(df_aug_train, embedder_tokenizer, config.max_len, config.batch_size)
notre_loader_test = create_dataloader(df_aug_test, embedder_tokenizer, config.max_len, config.batch_size)

In [15]:
#'new_heuristique_MJ' (PS: use gemma model only with this strategy)  
#'new_heuristique_LIG' or 
# "new_heuristique_MJ_by_lig_brute" or 
# "new_heuristique_MJ_our_annotation" or 
# "new_heuristique_MJ_combined_annotation"
# "new_heuristique_MJ_TCAVS"

joint_residual_model.run_full_pipeline_tcavs_strategy(
    train_loader=notre_loader_train,
    val_loader=val_loader,
    test_loader=notre_loader_test, 
    num_iterations=10, 
    num_epochs_residual_layer=5,
    cavs_type_arg = 'mean',
    strategy = 'new_heuristique_MJ_TCAVS', 
    coverage_threshold = 95
    )

new_heuristique_MJ_TCAVS
[(['players'], 78.6), (['industry-specific terminology and jargon'], 97.575), (['companies'], 97.725), (['economic trends'], 98.0), (['industries'], 98.05), (['summaries of events'], 99.9), (['technological trends'], 99.975), (['financial markets'], 99.975), (['industry analysis'], 99.975), (['Scores'], 99.975), (['charts, graphs, and financial data'], 99.975), (['advancements in computing'], 99.975), (['stock market updates'], 99.975), (['innovations'], 99.975), (['emerging technologies'], 99.975), (['new products'], 99.975), (['software'], 99.975), (['hardware'], 99.975), (['political developments'], 99.975), (['global issues'], 99.975), (['international events'], 99.975), (['global politics'], 99.975), (['foreign affairs'], 99.975), (['international relations'], 99.975), (['sports events'], 99.975), (['athletic competitions'], 99.975), (['jargon specific to the sport'], 100.0), (['game summaries'], 100.0), (['teams'], 100.0), (['analysis of global implicatio

Training:   6%|▌         | 14/250 [00:02<00:34,  6.89batch/s]


KeyboardInterrupt: 